In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
%cd drive/MyDrive/en2sparql

Mounted at /content/drive
/content/drive/MyDrive/en2sparql


In [ ]:
import torch
import json
from transformers import pipeline, AutoTokenizer

gpt2 = pipeline(model="gpt2-large", max_new_tokens=384, device='cuda' if torch.cuda.is_available() else "cpu", return_full_text=False)
tokenizer = AutoTokenizer.from_pretrained("gpt2-large")

In [ ]:

def divide_chunks(l_, n_):
    for i_ in range(0, len(l_), n_):
        yield l_[i_:i_ + n_]


def clean(st):
    st = st.replace("\n", " ")
    st = st.replace("?", " ?")
    st = st.replace("{", " { ")
    st = st.replace("}", " } ")
    st = st.replace("\\'", "'")

    while "  " in st:
        st = st.replace("  ", " ")
    return st


def load_json(file__name):
    data_file = open(file__name, "r", encoding='utf-8')
    file_data = json.loads(data_file.read())
    data_file.close()
    return file_data


def save_json(filename,data):
    with open(filename, "w", encoding="utf-8") as json_file:
        print(json.dumps(data), file=json_file)


def main(filename):
    all_data = load_json(filename)
    for key in all_data:
        data = all_data[key]
        q_list = data["questions"]
        suggestions = data["templates"]
        template = data["template"]
        gs = data["generated_sparql"]
        sparql = data["sparql"]
        lens = data.get("prompt_len")
        if lens is None:
            lens = []

        print(len(q_list))

        for i, question in enumerate(q_list[len(gs):]):
            print(i, end=" ")
            res_ = tokenizer.encode(question)
            len_ = len(res_)
            lens.append(len_)
            print("len: ",len_)

            if len_ > 600:
                question = tokenizer.decode(res_[-600:])
                len_ = 600

            res = gpt2(question)
            gs.append(res[0]["generated_text"])
            result = {"questions": q_list, "sparql": sparql, "generated_sparql": gs, "prompt_len": lens, "templates": suggestions, "template": template}
            all_data[key] = result
            save_json(filename, all_data)



In [ ]:
main("test_3_mult_diversity_pt_gpt2.json")